# ChatWithImage Module Usage Notebook

This notebook demonstrates how to use the `ChatWithImage` module to interact with a vision LLM.

**Prerequisites:**
- Install required packages
- Set the required environment variables (`GRPQ_APIKEY`), or pass them during initialization.

**Overview:**
- Initialize the `ChatWithImage` instance.
- Send a text prompt with or without an image.
- Use session-based chat to maintain conversation history.
- Stream tokens (if desired) from the LLM.
- Clear session history when needed.

Replace placeholder values with your actual credentials and file paths as required.

If you have the environment variables setup in .env file, you can load them using load_dotenv

In [ ]:
import sys
import os
from dotenv import load_dotenv
load_dotenv()

Optionally, set GROQ credentials here if they are not already set as environment variables.

In [ ]:
# os.environ["GROQ_APIKEY"] = "<your-groq-apikey>"

In [ ]:
# Add the parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

## 1. Initialize ChatWithImage

Import the module `ChatWithImage` and create an instance. You can customize the model parameters if needed.

In [ ]:
from utils.chat_image import ChatWithImage

In [ ]:
# Create a ChatWithImage instance.
chat_client = ChatWithImage(
    model_id="meta-llama/llama-4-maverick-17b-128e-instruct",
    max_tokens=1000,
    temperature=0
)
print("ChatWithImage client initialized!")

## 2. Chat without an Image (Text Only)

Use the `chat_with_image` method to send a text prompt.

If you do not provide a `session_id`, the conversation will be stateless (each call is independent).

In [ ]:
prompt_text = "Tell me about LLM in 1 sentence"
response = chat_client.chat_with_image(prompt=prompt_text)
print("Response:", response)

## 3. Chat with an Image

You can provide an image along with your prompt. Pass the image either as a file path, base64 or as a PIL image.

**Note:** The first time you send an image in a session, it is accepted; subsequent calls in the same session will raise an error if another image is provided.

In this example, we use an image file.

In [ ]:
image_path_1 = "../data/infographic.jpeg"
image_path_2 = "../data/infographic2.jpeg"
prompt_with_image = "How much time do we spend sleeping in our lifetime?"
# Here, we do not pass a session_id so the chat is stateless.
response_with_image = chat_client.chat_with_image(prompt=prompt_with_image, images=[image_path_1, image_path_2], convert_images_to_base64=True)
print("Response with image:", response_with_image)

## 4. Maintaining Session History

When interacting with the model over multiple turns, you can provide a `session_id` to maintain conversation history.

In the first call, you can also provide a system message (only allowed in the first message).
Subsequent messages in the same session should not include a system message.

In [ ]:
session_id = "my_chat_session"

# First message with a system prompt and optionally an image.
system_msg = "You are a helpful assistant that can also analyze images."
first_prompt = "How much time do we spend doing chores in our lifetime?"
first_response = chat_client.chat_with_image(
    prompt=first_prompt,
    images=[image_path_1, image_path_2],
    system_message=system_msg,
    session_id=session_id,
    convert_images_to_base64=True
)
print("First response (with session):", first_response)

In [ ]:
# Next message in the same session (without system message and image).
followup_prompt = "Summarize the contents of both the images"
followup_response = chat_client.chat_with_image(
    prompt=followup_prompt,
    session_id=session_id
)
print("Follow-up response (with session):", followup_response)

## 5. Streaming Tokens

To stream tokens as they are generated (instead of waiting for the full response), set the `stream` parameter to `True`.

When streaming, the method returns an iterator over token chunks.

In [ ]:
session_id_2 = "session2"

print("Streaming response:")
stream_iterator = chat_client.chat_with_image(
    prompt="Please describe the image in detail.",
    stream=True,
    images=[image_path_1],
    session_id=session_id_2
)

for token in stream_iterator:
    print(token.content, end="")

## 6. Retrieve Chat History

Use `get_chat_history` to fetch the conversation history for a specific session, and
`get_all_session_chat_history` to retrieve histories for all sessions.

In [ ]:
# Retrieve chat history for a specific session.
history = chat_client.get_chat_history(session_id)
print(f"Chat history for session '{session_id}':")
history

In [ ]:
# Retrieve chat histories for all sessions.
all_histories = chat_client.get_all_session_chat_history()
print("\nAll session chat histories:")
all_histories

## 6. Clearing Session History

If you wish to clear the conversation history for a given session (or all sessions), you can do so.

In [ ]:
# Clear a specific session history.
chat_client.clear_session_history(session_id)
print(f"Session history for '{session_id}' cleared.")

In [ ]:
# Clear all session histories.
chat_client.clear_all_session_histories()
print("All session histories cleared.")

In [ ]:
# Check chat histories after clearing all session histories to verify all sessions have been cleared
all_histories = chat_client.get_all_session_chat_history()
print("\nAll session chat histories:")
all_histories

# End of ChatWithImage Module Usage Notebook

This guide demonstrated:
- Initializing the ChatWithImage client.
- Chatting with text only or with an image.
- Maintaining conversation history with session ids.
- Streaming responses token-by-token.
- Retrieve session histories.
- Clearing session histories.

Modify parameters and prompts as needed for your application.